# Welcome to the first DataJoint webinar!

This webinar consists of three main parts. In the first part we will learn how to connect to a database. Afterwards the second part will teach us fundamental DataJoint usage which will be expanded upon in the third part when we will learn about working with dependent tables.

But first things first, let's start by connecting to a database:

## 1. Connecting to the DataJoint database

In this part we will learn how to...
1. ...import the DataJoint package
2. ...configure the connection information
3. ...test the connection
4. ...change the password of your database user

### 1.1 Importing the package

We must import the `datajoint` package before we can start using it. Convention is to alias the package to `dj`.

In [ ]:
import datajoint as dj

The `datajoint` package is now imported but a database connection has not yet been established. In order to establish a connection we first have to specify the connection information. We will learn how to do this in the next section.

### 1.2 Configuring the connection information

The configuration information DataJoint is currently using can be found in the `dj.config` object. Let's take a look:

In [ ]:
dj.config

The `dj.config` object behaves like a regular Python dictionary, i.e. it maps keys to values. The values associated with the three following keys tell DataJoint which database server it should connect to:
* `database.host` (The address of the database server)
* `database.user` (Your username)
* `database.password` (Your password)

For the purposes of this webinar the values are already correctly configured but usually you have to set them yourself.

### 1.3 Testing the connection

We can test our database connection by calling the `dj.conn` function directly:

In [ ]:
dj.conn()

The connection to the database server was successfully established if the above call returned without raising an exception.

### 1.4 Saving the connection configuration across sessions

The changes we make to the `dj.config` object are **not persistent** and we would need to redo them each time we start a new session which is not ideal.

Luckily DataJoint provides a way around this inconvenience by allowing us to save the current connection configuration for future sessions. We can use this feature by calling the `save_global` method of the `dj.config` object:

In [ ]:
dj.config.save_global()

This will create a file called `.datajoint_config.json` in your home directory. DataJoint will use the information contained within this file to configure itself automatically next time we import the `datajoint` package. Note that for the purposes of this webinar the global configuration file is already preconfigured.

Another way to store the connection information across sessions is to use the `save_local` method of the `dj.config` object. This method will store the information in a file in the current working directory, i.e. locally. Note that DataJoint can only use this information if the file is present in the current working directory when the `datajoint` package is imported.

### 1.5 Changing the password
The password you received for this webinar was randomly generated. Therefore it is a good idea to change the password to something that is easier to remember. You can change your password by calling the `dj.set_password` function:

In [ ]:
dj.set_password()

The function will automatically update the password in the `dj.config` object and save the new configuration locally if you answered yes to the corresponding question. The global configuration on the other hand is not automatically updated. Let's fix that:

In [ ]:
dj.config.save_global()

Congratulations! You successfully connected to our database server. Now it is time to learn the basics of DataJoint which we will do in the next section.
 
## 2. Learning basic DataJoint usage

### What is a data pipeline?
The documentation for [DataJoint](https://docs.datajoint.io/python/intro/01-Data-Pipelines.html?highlight=pipeline#what-is-datajoint) defines a data pipeline as follows:

> A data pipeline is a sequence of steps (more generally a directed acyclic graph) with integrated storage at each step.

While this is an accurate description, it may not be the most intuitive definition. Put succinctly, a data pipeline is a kind of "map" of various "things" that you work with in your project, with lines connecting the "things" to each other to indicate the dependecies between them. These "things" tend to be the nouns you use when describing your project to someone else. They may include things such as mouse, experimenter, session, trial, two-photon scan, electric activity, receptive field, neuronal spike, figure and so on.

A data pipeline gives you a framework to:
1. Define the "things" in your project as tables in which you can store information about the aforementioned "things"
2. Define the dependencies between the "things"

A data pipeline can then serve as a "map" that describes everything that goes on in your project, capturing what is collected and how it is processed. A well designed data pipeline not only lets you organize your data in an efficient way, but can visualize the logical clarity of your project. In doing so it may even bring about new insights by making the way everything in your project relates together obvious.

Let's go ahead and build a pipeline together from scratch to better understand what it's all about.

### Building your first pipeline: Single-electrode recording from mouse

Let's build a DataJoint pipeline to collect, store and process data for your hypothetical single-electrode recording study in mice. To help us understand the project better, here is a brief description:

> * Your lab houses many mice and each mouse is identified by a unique ID. You also want to keep track of information concerning each individual mouse such as its date of birth and gender.
> * As a hard working neuroscientist, you perform experiments daily, sometimes working with more than one mouse in a day! However, on any given day, a mouse undergoes at most one recording session.
> * For each experimental session you would like to record which mouse you worked with and when you performed the experiment. You would also like to keep track of other helpful information such as which experimental setup you worked on.
> * In each experimental session you record electrical activity from a single neuron and you use recording equipment that produces separate data files for each recorded neuron.
> * The activities of the neurons are recorded as raw traces and the spikes contained within the traces need to be detected for further analysis.

Pipeline design starts by identifying "things" (a.k.a. entities) in your project. Common "things" may include experimental subjects (e.g. mouse), recording sessions and two-photon scans.

Let's revisit the project description, this time paying special attention to the "things" (i.e. nouns) in your project. Here I have highlighted some nouns that might be interesting:

> * Your lab houses many mice and each **mouse** is identified by a unique ID. You also want to keep track of information concerning each individual mouse such as its date of birth and gender.
> * As a hard working neuroscientist, you perform experiments daily, sometimes working with more than one mouse in a day! However, on any given day, a mouse undergoes at most one recording session.
> * For each **experimental session** you would like to record which mouse you worked with and when you performed the experiment. You would also like to keep track of other helpful information such as which experimental setup you worked on.
> * In each experimental session you record electrical activity from a single **neuron** and you use recording equipment that produces separate data files for each recorded neuron.
> * The activities of the neurons are recorded as raw traces and the **spikes** contained within the traces need to be detected for further analysis.

Just by going through the description, we can start to identify "things" that we might want to store and represent in your data pipeline:

* mouse
* experimental session
* neuron
* spikes

In a DataJoint data pipeline, we represent these "things" as tables. Different kinds of "things" are represented by distinct tables and each row of a given table contains a single entry of the kind that the table represents.

For example, if we have a table representing mice, then each row in that table represents an individual mouse!

When constructing such a table, we need to figure out what information is necessary to uniquely identify any given entry of the kind represented by the table. Let's take the example of the mouse and think about what it would take to uniquely identify a mouse.

After some thought we might conclude that each mouse can be uniquely identified by its ID - a unique number assigned to each mouse. Thereafter follows that the ID should be an attribute (i.e. a column) in the table that can be used to uniquely identify each mouse. Such uniquely identifying attributes are called the **primary key** of the table.

| mouse_id* |
|:--------- |
| 11234     |
| 11432     |

Once we have successfully identified the primary key of the table, we can think about what other **non-primary attributes** we want to include in the table. These will contain additional information about each entry in the table.

In the case of the mouse table, what other information about each individual mouse would you might want to store? Based on the project description, we would probably want to store information such as the mouse's date of birth and gender.

| mouse_id* | dob	     | sex |
|:--------- | ----------:| ---:|
| 11234	    | 2017-11-17 |	M  |
| 11432	    | 2018-03-04 |	F  |

Now that we have an idea of how to represent information about the mice in your project, let's start creating the table using DataJoint!

### 2.1 Creating a schema

To start building your pipeline, we first need to define a place for your tables to life in. In DataJoint this place is called a schema which is a logical collection of one or more tables. Any given pipeline might consist of many tables spread across one or more schemas.

We can create a schema by passing the name of the schema to DataJoint's `Schema` class. This will return a schema object which we can assign to a local variable (called `schema` by convention) for later use. Note that DataJoint's `Schema` class is aliased as `dj.schema` in the current context. Also note that you are only allowed to create/access schemas starting with your username followed by a single underscore (e. g. `"john_pipeline"` if your username is `"john"`). Let's go ahead and create a schema for your first table to live in:

In [ ]:
# ENTER YOUR CODE! - create your own schema object

### 2.2 Creating a table

DataJoint uses Python classes to represent tables. To be a valid DataJoint table, a class needs to...
1. ...inherit from one of DataJoint's base classes (e.g. `dj.Manual`)
2. ...define a class attribute of type `str` called `definition`
3. ...be decorated with an instance of DataJoint's `Schema` class

Remember that each table class is associated with a table in the database and note that the database table is created once the table class is decorated with the schema instance.

Let's go ahead and create the first table in our pipeline:

In [ ]:
# ENTER YOUR CODE! - complete your first table definition


class Mouse:
    pass

Note that changing the table class (e.g. changing attribute types) after the database table was created will not change it.

Let's take a look at your new table:

In [ ]:
Mouse()

Right now the table is pretty boring because it is completely empty. Let's make it more interesting by inserting some entries into it. We will learn how to do this in the next section.

### 2.3 Inserting entries

We can insert a single entry into a table by calling the table's `insert1` method with the entry that we want to insert packaged as a sequence (e.g. a `tuple`). Let's try to insert a new mouse into the `Mouse` table using the following information:
* ID: 0
* Date of birth: 2017-03-01
* Sex: male

In [ ]:
Mouse.insert1((0, "2017-03-01", "M"))

In [ ]:
Mouse()

We can also insert an entry using a mapping (e.g. a `dict`) instead of a sequence:

In [ ]:
data = {
    "mouse_id": 100,
    "dob": "2017-05-12",
    "sex": "F",
}

In [ ]:
# ENTER YOUR CODE! - insert a single mapping

In [ ]:
Mouse()

Inserting individual entries one by one will get tiresome if we have a lot of them. Luckily we can insert multiple entries at once using the table's `insert` method, passing in a sequence of sequences (e. g. a `list` of `tuples`):

In [ ]:
data = [
    (1, "2016-11-19", "M"),
    (2, "2016-11-20", "unknown"),
    (5, "2016-12-25", "F"),
]

In [ ]:
# ENTER YOUR CODE! - insert multiple entries at once

In [ ]:
Mouse()

Just as before with the `insert1` method, we can pass mappings instead of sequences to the `insert` method as well:

In [ ]:
data = [
    {"mouse_id": 10, "dob": "2017-01-01", "sex": "F"},
    {"mouse_id": 11, "dob": "2017-01-03", "sex": "F"},
]

Mouse.insert(data)

In [ ]:
Mouse()

DataJoint does not allow the insertion of an entry that has the same primary key as an already existing one because that would violate the integrity of the data. Let's test this feature by trying to insert a mouse that is already present in the table:

In [ ]:
Mouse.insert1(
    {
        "mouse_id": 0,
        "dob": "2018-01-01",
        "sex": "M",
    }
)

#### Exercise
Insert a single **male** mouse with **ID 6** that was born on **2017-04-15**:

In [ ]:
# ENTER YOUR CODE!

Insert **two** mice at the same time. The first one should have **ID 7**, be **female** and its date of birth should be **2016-10-06**. The second one should have **ID 8**, be **male** and its date of birth should be **2016-10-08**:

In [ ]:
# ENTER YOUR CODE!

Excellent! Now we know how to insert entries into the table but what about getting data out of the table? This process is called "fetching" in the database world and we will learn how to do it in the next section.

### 2.4 Fetching data

We can fetch all entries in the table by calling the table's `fetch` method:

In [ ]:
Mouse.fetch()

The `fetch` method returns the entries as a structured Numpy array by default. We can tell it to return the entries as a list of dictionaries instead by passing the keyword argument `as_dict` with a value of `True` to `fetch`:

In [ ]:
# ENTER YOUR CODE! - fetch as a list of dictionaries

What if we only want to fetch a specific attribute instead of all of them? We can do that by passing the names of the attributes that we want to the `fetch` method:

In [ ]:
# ENTER YOUR CODE! - fetch the "sex" attribute of all mice

We can also fetch multiple attributes by passing additional attribute names to `fetch`:

In [ ]:
sex, dob = Mouse.fetch("sex", "dob")

In [ ]:
sex

In [ ]:
dob

We can pass `"KEY"` to `fetch` if we just want the primary keys of all entries:

In [ ]:
data = Mouse.fetch("KEY")
data

Note that this will always return the primary key attributes as dictionaries.

Fetching all entries as a Pandas DataFrame is also possible:

In [ ]:
data = Mouse.fetch(format="frame")
data

Fetching all data at once is pretty unefficient in most scenarios. Luckily DataJoint offers a way to query specific data and we will learn all about it in the next section.

### 2.5 Querying data

Often times we want to work with **a subset of entries** that match a specific condition and not all entries. Using DataJoint's intuitive **querying** syntax lets us select only the entries that we want before fetching them.

#### 2.5.1 Restricting by attribute value

The **restriction** operator `&` uses the condition to the right of the operator to narrow the entries in the table to the left of the operator down to the ones fulfilling the condition.

Syntax: `table & condition`

##### 2.5.1.1 Equality

Let's restrict the mouse table down to the mouse with **ID 0**:

In [ ]:
Mouse & "mouse_id = 0"

All **male** mice:

In [ ]:
# ENTER YOUR CODE!

##### 2.5.1.2 Inequality

All mice born **after 2017-01-01**:

In [ ]:
# ENTER YOUR CODE!

All mice that are **not male**:

In [ ]:
# ENTER YOUR CODE!

#### 2.5.2 Combining restrictions

We can combine multiple restrictions to narrow down the set of entries based on multiple attributes.

Let's find all mice that are **not male** AND were **born after 2017-01-01**:

In [ ]:
# ENTER YOUR CODE!

The result of one query can be used in another query. Let's find **all female mice** and store the result:

In [ ]:
female_mice = Mouse & "sex = 'F'"
female_mice

And among these mice, find the ones with **mouse_id > 10**:

In [ ]:
female_mice & "mouse_id > 10"

#### 2.5.3 Restricting with a dictionary

We can also use a dictionary for restrictions. This approach can improve the readability of our code if we would need to chain a lot of string restrictions otherwise. Let's see what querying all male mice that were born on 2016-11-19 looks like using a dictionary:

In [ ]:
restriction = {
    "sex": "M",
    "dob": "2016-11-19",
}
Mouse & restriction

### 2.6 Fetching queried entries

We can fetch the queried entries by simply calling the `fetch` method on the query result. Let's try to **fetch all male mice**:

In [ ]:
male_mice = Mouse & "sex = 'M'"
male_mice

Fetch them!

In [ ]:
# ENTER YOUR CODE!

We can also query and fetch in one expression:

In [ ]:
(Mouse & "sex = 'M'").fetch()

### 2.7 Deleting data

Now that we have a good understanding of how to restrict tables, we can look at how to delete entries from tables.

To delete a specific entry, we need to restrict the table down to the target entry and then call the `delete` method on the restricted table:

In [ ]:
(Mouse & "mouse_id = 100").delete()

Calling the `delete` method on an unrestricted table will attempt to delete all entries from the table:

In [ ]:
Mouse.delete()

#### Exercise
Fetch all mice born in **November of 2016**:

In [ ]:
# ENTER YOUR CODE!

This concludes part two of the webinar. Next we will look at working with dependent tables:

## 3. Working with dependent tables

Congratulations! We have successfully created your first table and in doing so we learned how to insert, query, fetch and delete data! We are now ready to integrate other "things" from the project into your data pipeline.

Let's now take a look at representing an experimental session.

As with representing a mouse, we should first think about **what information (i.e. which attributes) is needed to uniquely identify an experimental session**. Here is the relevant section of the project description:

> * As a hard working neuroscientist, you perform experiments daily, sometimes working with **more than one mouse in a day**! However, on any given day, **a mouse undergoes at most one recording session**.
> * For each experimental session you would like to record **which mouse you worked with** and **when you performed the experiment**. You would also like to keep track of other helpful information such as **which experimental setup** you worked on.

Based on the above, it appears that you need to know:

* The date of the session
* The mouse you recorded from in that session

to uniquely identify a single experimental session.

Note that, to uniquely identify an experimental session (or simply a session), we need to know which mouse you recorded from in that session. In other words a session cannot existing without a corresponding mouse!

With a table representing mice already present in our data pipeline, we say that the session table **depends on** the mouse table! We would graphically represent this in an **entity relationship diagram (ERD)** by drawing a line between the two tables, with the one below (**session**) depending on the one above (**mouse**).

![mouse-session](../figures/mouse-session.svg)

Thus we will need both an existing mouse entry and a new attribute called `session_date` to uniquely identify a single session.

Remember that a mouse is already uniquely identified by its primary key - `mouse_id`. In DataJoint you can declare that the session table depends on the mouse table and DataJoint will automatically include the primary key of the mouse table (i.e. `mouse_id`) as part of the session table's primary key, along side any additional attribute(s) you specify.

### 3.1 Defining a dependent table

In [ ]:
@schema
class Session(dj.Manual):
    definition = """
    # experimental session
    -> Mouse
    session_date       : date          # session date
    ---
    experiment_setup   : int           # experiment setup ID
    experimenter       : varchar(100)  # name of the experimenter
    start              : time          # starting time of the session
    end                : time          # ending time of the session
    """

We can create the entity relationship diagram (ERD) by calling `dj.Diagram`, passing the `schema` object:

In [ ]:
# ENTER YOUR CODE! - plot the ERD of the schema

Let's try to insert your first session manually:

In [ ]:
data = {
    "mouse_id": 0,
    "session_date": "2017-05-15",
    "experiment_setup": 0,
    "experimenter": "Edgar Y. Walker",
    "start": "10:22:34",
    "end": "10:54:32",
}
Session.insert1(data)
Session()

Let's try to insert another session for the same mouse (i.e. ID 0) that was done on a different date:

In [ ]:
data = {
    "mouse_id": 0,
    "session_date": "2018-01-15",
    "experiment_setup": 100,
    "experimenter": "Christoph Blessing",
    "start": "12:32:43",
    "end": "13:15:29",
}
Session.insert1(data)
Session()

And another session that was done on the same date as the one before but on a different mouse:

In [ ]:
data = {
    "mouse_id": 11,
    "session_date": "2018-01-15",
    "experiment_setup": 101,
    "experimenter": "Christoph Blessing",
    "start": "8:58:42",
    "end": "9:36:40",
}
Session.insert1(data)
Session()

Let's see what happens if we try to insert a session for which no mouse exists:

In [ ]:
bad_data = {
    "mouse_id": 9999,
    "session_date": "2017-05-15",
    "experiment_setup": 0,
    "experimenter": "Edgar Y. Walker",
    "start": "19:26:51",
    "end": "21:32:12",
}
Session.insert1(bad_data)
Session()

### 3.2 Querying with multiple tables

Now that we have two tables, we can perform more exciting queries!

#### 3.2.1 Restricting by another table

Find all mice that had a session performed on them:

In [ ]:
# ENTER YOUR CODE!

#### 3.2.2 Combining restrictions

Let's try to find all sessions performed on male mice:

In [ ]:
# ENTER YOUR CODE!

Let's try to find all mice that had an experimental session done on or before 2017-05-19:

In [ ]:
# ENTER YOUR CODE!

##### Exercise
Fetch all mice that had a session performed on them **before 11 a.m.**

In [ ]:
# ENTER YOUR CODE!

### 3.3 Deleting entries that have dependent entries

Let's see what happens if we try to delete a mouse that had sessions performed on it:

In [ ]:
(Mouse & "mouse_id = 0").delete()

Note that when deleting an entry (e.g. a mouse) DataJoint will automatically cascade the deletion process down to any dependent entries (e.g. sessions) and delete them as well. This ensures that no orphaned entries are present in the data pipeline after an entry has been deleted.

### 3.4 Joining tables

Sometimes it is useful to have entries from multiple tables combined together to be viewed and queried. You can accomplish this using the join `*` operator.

Let's look at the combination of all mice and sessions:

In [ ]:
Mouse * Session

Each row represents a unique and valid combination of a mouse and a session.

The combined table can be queried using any of the attributes present in the joined tables.

Let's try to find all combinations where Edgar Y. Walker was the experimenter and the sex of the mouse was male:

In [ ]:
# ENTER YOUR CODE!

### 3.5 Projecting tables

Tables can be projected to select a subset of their attributes, to rename attributes or to calculate new attributes from existing ones.

#### 3.5.1 Selecting attributes

We can project the `Mouse` table onto its primary key (i. e. `mouse_id`) by calling the table's `proj` method without any arguments:

In [ ]:
Mouse.proj()

We can select additional attributes besides the primary key by passing their names to the `proj` method.

Let's try to project the `Session` table onto the `start` and `end` attributes:

In [ ]:
# ENTER YOUR CODE!

#### 3.5.2 Renaming attributes

We can rename existing attributes by passing keyword arguments of the form `new_name=current_name` to the `proj` method.

Let's try to rename the `dob` attribute of the `Mouse` table to `date_of_birth`:

In [ ]:
Mouse.proj(date_of_birth="dob")

Renaming is often used to create combinations of different entries in a table. Let's try to create all unique combinations of date of births between the mouse with ID 0 and all other mice:

In [ ]:
# ENTER YOUR CODE!

#### 3.5.3 Calculating new attributes

A powerful feature of projection is the ability to calculate new attributes from existing ones. 

Let's say we want to calculate the duration of each session and restrict the output to sessions that were shorter than 60 minutes. First we calculate the session durations using the `proj` method and store the resulting projection in a variable:

In [ ]:
durations = Session.proj(duration="(end - start) / 60")  # Result of substraction is in seconds
durations

Then we can restrict our previously created projection to get all the sessions that were shorter than 60 minutes.

In [ ]:
durations & "duration < 60"

Calculations are not parsed by DataJoint but instead passed to SQL. This means we can use SQL functions in the computations!

Let's say we want to calculate the age of the mouse we worked with in each session at the date the session was performed using SQL's `DATEDIFF` function:

In [ ]:
(Mouse * Session).proj(age="DATEDIFF(session_date, dob)")  # Age is in days

## 4. Summary

Congratulations on completing your first DataJoint webinar.

We learned how to...
1. ...connect to a database
2. ...create schemas and tables
3. ...insert entries
4. ...fetch data
5. ...query data
6. ...delete entries
7. ...work with dependent tables
8. ...join tables
9. ...project tables

We are looking forward to seeing you in the next DataJoint webinar!